In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
"""
# Preprocesado y modelado
# ==============================================================================
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')"""


Cargamos los datos

In [ ]:
data = pd.read_csv('attrition_availabledata_35.csv.gz', compression="gzip")

#Informción general
data.info()
#Variable objetivo
data.Attrition.value_counts().sort_index() #Número de muestras por clase

Datos de las variables numéricas

In [ ]:
data.describe(include="all")

In [ ]:
print(data.keys())

Empezamos borrando las columnas que aportan datos irrelevantes

In [ ]:
#Variable a estudiar BussineTravel
print(data["BusinessTravel"].value_counts()) #Parece que tiene valores distribuidos, por lo que la vamos a dejar

#Variable a estudiar EmployeeCount
print(data["EmployeeCount"].value_counts())  #Solo tiene un único valor, por lo que es una variable descartable

#La variable EmployeeID es irrelevante en el modelo, por lo que la descartamos igual

#Variable a estudiar Over18
print(data["Over18"].value_counts()) #Único valor --> descartable

#Variable a estudiar StandardHours
print(data["StandardHours"].value_counts()) #Único valor --> descartable

#Variable a estudiar TrainigTimesLastYears
print(data["TrainingTimesLastYear"].value_counts()) #Valores Distribuidos

#Vamos a eliminar EmployeeCount, Over18, StandardHours y EmployeeID  por ser irrelevante o tener un único valor


In [ ]:
# Lista de columnas a eliminar
cols_to_drop = ["EmployeeID", "EmployeeCount", "Over18", "StandardHours"]

# Eliminamos las columnas del dataset
data = data.drop(columns=cols_to_drop)

# Verificamos
data.info()

Tenemos ahora 26 variables a estudiar. Vamos hacer una matriz de correlación para ver si podemos descartar algunas más. Primero tendremos que filtrar los valores numéricos

In [ ]:
data["Attrition"] = data["Attrition"].map({"Yes": 1, "No": 0})
numeric_data = data.select_dtypes(include=["number"])

corr_matrix = numeric_data.corr()
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap (corr_matrix, linewidths = 0.5, annot = True, fmt= '.1f',ax=ax)
plt.show ()

In [ ]:
"""
Existen dos parejas de variables con una correlación de 0.8 (PercentSalaryHike/PerformanceRating y YearsInCompany/YearsWithCurrentMananger)
Voy a descartar una variable de cada pareja, en este caso PerformanceRating y YearsAtCompany
"""

# Lista de columnas a eliminar
cols_to_drop = ["PerformanceRating", "YearsAtCompany"]

# Eliminamos las columnas del dataset
data = data.drop(columns=cols_to_drop)

# Verificamos
data.info()

Ahora queremos descartar aquuelas variables que estan poco correlacionadas con la variable objetivo

In [ ]:

corr_with_target = numeric_data.corr()["Attrition"].sort_values(ascending=False)
#Mostramos las variables menos correlacionadas
print(corr_with_target.head(24))

Vamos a quitar aquellas que tengan una correlación menor a |0.03| 

In [ ]:
# Definir el umbral de correlación
threshold = 0.03  

# Recalcular la matriz de correlación con las columnas actuales del dataset
corr_matrix = data.select_dtypes(include=["number"]).corr()

# Obtener la correlación con Attrition
corr_with_target = corr_matrix["Attrition"].abs()

# Identificar columnas con correlación menor a 0.03 (demasiado bajas)
columns_to_drop = corr_with_target[corr_with_target < threshold].index

# Filtrar solo las columnas que realmente existen en el DataFrame antes de eliminarlas
columns_to_drop = [col for col in columns_to_drop if col in data.columns]

# Eliminar las columnas con baja correlación
data_filtered = data.drop(columns=columns_to_drop)

# Mostrar las columnas restantes
print("Columnas que se mantienen:", data_filtered.columns.tolist())
